In [177]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import torch 
import torch.nn as nn
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Some Helper Functions

In [178]:
def printUniqueCounts(df, columnName: str = None):
    column = None
    if columnName is None:
        column = df
    else:
        column = df[columnName]
    titles = column.unique()
    counts = []
    for title in titles:
        if pd.isna(title):
            count = sum(column.isnull())
        else:
            count = sum(column==title)
        counts.append([title,count])
    
    counts.sort(key=lambda x: -x[1])
    
    for item in counts:
        print(str(item[0])+": "+str(item[1]))

def torchEnumerator(tensor):
    array = tensor.detach().numpy()
    print("shape" + str(array.shape))
    return np.nditer(array)

In [179]:
trainData = pd.read_csv('train.csv')
print(trainData.shape)
trainData.head()


(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Now it's time to clean up the data for use with our ML algorithm and do some feature engineering.

First lets see which columns have NaN / null values:

In [180]:
for column in trainData.columns:
    empty = len(trainData[trainData[column].isnull()])
    if empty>0: print(column + " has " + str(empty) + " missing data points")

Age has 177 missing data points
Cabin has 687 missing data points
Embarked has 2 missing data points


We will fill in missing age values with average age for now. In the future we may want to infer the age from the other columns, but for now this will have to suffice.

In [181]:
#fill in missing age values with average age
avgAge = trainData["Age"].mean()
print("Average age: ")
trainData["Age"].fillna(avgAge)

Average age: 


0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64

To gain a better idea of how to handle the missing values in embarked, lets first see how many people embarked from each port:

In [182]:
printUniqueCounts(trainData,"Embarked")

S: 644
C: 168
Q: 77
nan: 2


There are only 3 ports of embarcation, which makes this feature a good candidate for one-hot encoding. Missing values will simply be recorded as 'False' in all embarcation ports

In [183]:
trainData = pd.get_dummies(trainData,columns=["Embarked"])
trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,False,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,True,False,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,False,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,False,False,True
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,False,False,True


The remaining column with missing data is Cabin. This is really two features: cabin area (the letter) and cabin number (the integer). For the sake of simplicity, lets assume the cabin number is not useful, but that the cabin area is. This is a reasonable assumption because the cabin area is likely to have a large effect on the location of the passenger, whereas the cabin number is less likely to have a large effect.

In [184]:

def getCabinLetter(cabin: str):
    if cabin is None or not isinstance(cabin,str): return None
    result =  re.search("^[a-zA-Z]*",cabin)
    if not result: return None
    return result[0]
    
cabinLetter = trainData["Cabin"].apply(getCabinLetter)

print("Cabin Letters")
printUniqueCounts(cabinLetter)


Cabin Letters
None: 687
C: 59
B: 47
D: 33
E: 32
A: 15
F: 13
G: 4
T: 1


We can see there are only 8 valid options for cabin letter. Being a small number this is a good candidate for one-hot encoding, again solving the missing data issue.

In [185]:
trainData["Cabin"] = cabinLetter
trainData = pd.get_dummies(trainData,columns=["Cabin"])
trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,False,True,False,False,False,False,False,False,False,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,False,False,False,False,True,False,False,False,False,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,False,True,False,False,False,False,False,False,False,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,False,True,False,False,True,False,False,False,False,False
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,False,True,False,False,False,False,False,False,False,False


That covers all the missing data, so now we just need to deal with non-numeric data. Lets start by converting Male/Female and True/False to 0/1

In [186]:
trainData["Sex"] = trainData["Sex"]=="male"
trainData = trainData*1
trainData.head()
printUniqueCounts(cabinLetter)

None: 687
C: 59
B: 47
D: 33
E: 32
A: 15
F: 13
G: 4
T: 1


Name and Ticket remain as non-numeric data. We probably could do some NLP to make useful inferences out of the names, but that would be a lot of work and likely not yield a big difference in results. Ticket is also likely not to be very useful info. So lets just delete both columns.

In [187]:
trainData = trainData.drop(columns=["Name","Ticket","PassengerId"])
trainData.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T
0,0,3,1,22.0,1,0,7.2500,0,0,1,0,0,0,0,0,0,0,0
1,1,1,0,38.0,1,0,71.2833,1,0,0,0,0,1,0,0,0,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1,0,0,0,0,0,0,0,0
3,1,1,0,35.0,1,0,53.1000,0,0,1,0,0,1,0,0,0,0,0
4,0,3,1,35.0,0,0,8.0500,0,0,1,0,0,0,0,0,0,0,0


Finally lets do a little bit of feature engineering. Most of these features look pretty useful as is, but Sibsp and Parch stand out as being a potentially useful target for a little bit of engineering. 
SibSp: siblings and spouses
Parch: parents and children

We can combine these two to get family size. Total family size might be an important metric so lets make an additional column for family size 

In [188]:

trainData.insert(len(trainData.columns),"FamSize",trainData["SibSp"]+trainData["Parch"])
trainData.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,FamSize
0,0,3,1,22.0,1,0,7.2500,0,0,1,0,0,0,0,0,0,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0,0,0,1,0,0,0,0,0,1
2,1,3,0,26.0,0,0,7.9250,0,0,1,0,0,0,0,0,0,0,0,0
3,1,1,0,35.0,1,0,53.1000,0,0,1,0,0,1,0,0,0,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1,0,0,0,0,0,0,0,0,0


Finally we have to normalize the data. Lets use zscore normalization on Age and Fare:
**temporarily dropping Age to fix bugs**

In [189]:
from scipy.stats import zscore
colsToNormalize = ["Age","Fare"]
trainData[colsToNormalize] = trainData[colsToNormalize].apply(zscore)

#drop age for now

trainDataY = trainData["Survived"]
trainDataX = trainData.drop(columns=["Survived","Age"])
trainDataX.head()




,Pclass,Sex,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,FamSize
0,3,1,1,0,-0.502445,0,0,1,0,0,0,0,0,0,0,0,1
1,1,0,1,0,0.786845,1,0,0,0,0,1,0,0,0,0,0,1
2,3,0,0,0,-0.488854,0,0,1,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0.420730,0,0,1,0,0,1,0,0,0,0,0,1
4,3,1,0,0,-0.486337,0,0,1,0,0,0,0,0,0,0,0,0


Now we have prepared the data for our ML algorithm.

Lets create a model with 3 hidden layers, of 20, 8 and 5 neurons. Each hidden layer will have a ReLU activation function. The final output neuron will have a sigmoid activation function.
For the loss function we will use binary cross entropy.

In [195]:
inCount = trainDataX.shape[1]#number of inputs
layerWeights = []
layerBiases = []

relu = nn.ReLU()
sigmoid = nn.Sigmoid()
layers = [[20,relu],[8,relu],[5,relu],[1,sigmoid]]#[node count, activation function]
layerCount = len(layers)
loss = nn.BCELoss()
learningRate = 0.01
initialParameterMin = -.1
initialParameterMax = .1

lastNodeCount = inCount
for layer in layers:
    nodeCount = layer[0]
    weights = torch.FloatTensor((lastNodeCount,nodeCount))
    biases = torch.FloatTensor(nodeCount)

    for arr in [weights,biases]:
        arr.uniform_(initialParameterMin,initialParameterMax)
        arr.requires_grad_(True)


    layerWeights.append(weights)
    layerBiases.append(biases)
    lastNodeCount = nodeCount

def printNan(V):
    nanCount = 0
    for item in torchEnumerator(V):
        if math.isnan(item): nanCount+=1
    ratio = nanCount/len(V)
    print(str(ratio*100)+"% Nan")

def forward(X):
    A = X
    print(X.shape)
    for i in range(layerCount):
        layer = layers[i]
        function = layer[1]
        prod = torch.matmul(A,layerWeights[i])
        z = prod + layerBiases[i]
        A = function(z)
    A = A.squeeze()
    return A
        
        
X = torch.tensor(trainDataX.values,dtype=torch.float32)
Y = torch.tensor(trainDataY.values,dtype=torch.float32)

losses = []
iterations = 100
for epoch in range(iterations):
    yHat = forward(X)
    l = loss(Y,yHat)
    l.backward()
    print(l)
    print(sum(weights.grad))
    with torch.no_grad():
        for i in range(layerCount):
            weights = layerWeights[i]
            biases = layerBiases[i]
            weights-=weights.grad*learningRate
            biases-=biases.grad*learningRate
            weights.grad.zero_()
            biases.grad.zero_()

    
    losses.append(l)


print(losses)

torch.Size([891, 17])


RuntimeError: size mismatch, got input (891), mat (891x17), vec (2)